In [ ]:
import numpy as np
from common.functions import cross_entropy_error, softmax

# 誤差逆伝播法

### 計算グラフ
- 複雑な計算も，単純な関数の合成関数とみなすことができる
#### 連鎖律
- ある関数が合成関数で表される場合，その合成関数の微分は，合成関数を構成するそれぞれの関数の微分の積によって表すことができる
$$
    z=f(t), t=f(x)の時，\frac{\sigma z}{\sigma x} = \frac{\sigma z}{\sigma t} \frac{\sigma t}{\sigma x}
$$

加算ノード，乗算ノード，活性化関数など，それぞれの場合の逆伝播がわかれば，簡単に数値微分を計算できる！

In [ ]:
###単純なレイヤの実装###

class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    def forward(self, x, y):
        self.x, self.y = x, y
        return x*y
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x  #乗算ノードの逆伝播はひっくり返してかけるだけ(z=x*y: dz/dx=y, dz/dy =x)
        return dx, dy

class AddLayer:
    def __init__(self):
        pass
    def forward(self, x, y):
        return x+y
    def backward(self, dout):
        return dout, dout       #加算ノードでは逆伝播は変化しない(z=x+y: dz/dx=1, dz/dy =1)
    
###活性化関数レイヤの実装###
class Relu:
    def __init__(self):
        self.mask = None
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx
    
class Sigmoid:
    def __init__(self):
        self.out = None
    def forward(self, x):
        self.out = 1/(1+np.exp(-x))
        return self.out
    def backward(self, dout):
        dx = dout * (1-self.out)*self.out
        return dx

In [ ]:
###Affineレイヤの実装(Y=XW+Bの計算をするレイヤ)###
"""
class Affine:
    def __init__(self, W, b):
        self.W , self.b = W, b
        self.x = self.dW = self.db = None
    def forward(self, x):
        self.x = x
        return np.dot(x, self.W)+self.b
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)

        return dx
"""
class Affine:
    def __init__(self, W, b):
        self.W =W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        # 重み・バイアスパラメータの微分
        self.dW = None
        self.db = None

    def forward(self, x):
        # テンソル対応
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)  # 入力データの形状に戻す（テンソル対応）
        return dx
    
###Softmaxレイヤの実装(損失関数付き)###
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = self.y = self.t = None
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    def backward(self, dout =1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: # 教師データがone-hot-vectorの場合
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx


### 誤差逆伝播法の実装

In [ ]:
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        self.params = {}
        self.params["W1"] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params["b1"] = np.zeros(hidden_size)
        self.params["W2"] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params["b2"] = np.zeros(output_size)

        self.layers = OrderedDict()
        self.layers["Affine1"] = Affine(self.params["W1"], self.params["b1"])
        self.layers["Relu1"] = Relu()
        self.layers["Affine2"] = Affine(self.params["W2"], self.params["b2"])
        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.laseLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1: t=np.argmax(t, axis=1)

        accuracy = np.sum(y==t) / float(x.shape[0])
        return accuracy       

    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t) 

        grads = {}
        grads["W1"] = numerical_gradient(loss_W, self.params["W1"])
        grads["W2"] = numerical_gradient(loss_W, self.params["W2"])
        grads["b1"] = numerical_gradient(loss_W, self.params["b1"])
        grads["b2"] = numerical_gradient(loss_W, self.params["b2"])

        return grads
    
    def gradient(self, x, t):
        self.loss(x, t)

        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        grads = {}
        grads["W1"] = self.layers["Affine1"].dW
        grads["W2"] = self.layers["Affine2"].dW
        grads["b1"] = self.layers["Affine1"].db
        grads["b2"] = self.layers["Affine2"].db

        return grads




In [ ]:
from keras.datasets import mnist
from keras.utils import to_categorical

(x_train, t_train), (x_test, t_test) = mnist.load_data()
t_train = np.eye(N=10)[t_train.astype('int32').flatten()]   #どちらも同じ処理．one hot labelにしてる
t_test = to_categorical(t_test)
x_train = x_train.astype("float32")/255.


iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

x_train = x_train.reshape([x_train.shape[0], 784])
x_test = x_test.reshape([x_test.shape[0], 784])
print(x_train.shape)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size/batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)

    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    grad = network.gradient(x_batch, t_batch)

    for key in ("W1", "b1", "W2", "b2"):
        network.params[key] -= learning_rate*grad[key]

    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i%iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)
    
